<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/XGBoost_and_AdaBoost_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.metrics import make_scorer, confusion_matrix, classification_report, precision_score, r2_score, f1_score, recall_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

import time

### Mount data from Google Drive

In [4]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Change dir and check contents

In [5]:
cd '/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput'

/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput


In [6]:
ls

winner_matches_agg.csv  winner_side_rounds_agg.csv  winner_team_rounds_agg.csv


### load data


In [7]:
%%time
pred_team_matches = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_matches_agg.csv')
pred_team_rounds = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_side_rounds_agg.csv')
pred_side_rounds = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_team_rounds_agg.csv')

CPU times: user 1.23 s, sys: 263 ms, total: 1.49 s
Wall time: 2.56 s


In [8]:
pred_team_matches

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,25,84.747674,20009.375000,1,0,0,0,0,0,0,0,-1
1,39,97.655995,21298.717949,0,1,0,0,0,0,0,0,-1
2,29,94.112689,18600.000000,0,0,1,0,0,0,0,0,-1
3,30,94.452687,19579.166667,0,0,0,1,0,0,0,0,1
4,29,102.916448,18168.103448,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,29,81.956562,18927.586207,0,1,0,0,0,0,0,0,1
13457,24,74.316426,18516.304348,0,0,0,0,0,1,0,0,1
13458,18,87.696972,16775.000000,0,0,1,0,0,0,0,0,1
13459,24,75.069213,18711.458333,0,0,0,0,1,0,0,0,1


In [9]:
pred_team_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class
0,1,94.30782,160.9591,66.65128,4300,4250,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1


In [10]:
pred_side_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_class
0,1,94.30782,160.9591,66.65128,4300,4250,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,1


### apply standard scaler

In [11]:
stdscl = StandardScaler()
stdscl

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
pred_team_matches.columns

Index(['num_rounds', 'avg_round_duration', 'avg_equip_val', 'de_overpass',
       'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2',
       'de_cbble', 'de_nuke', 'winner_class'],
      dtype='object')

In [13]:
std_pred_team_matches = pd.DataFrame(data=StandardScaler().fit_transform(pred_team_matches[['num_rounds', 'avg_round_duration', 'avg_equip_val']]),columns=['num_rounds', 'avg_round_duration', 'avg_equip_val'])
std_pred_team_matches[['de_overpass','de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2','de_cbble', 'de_nuke', 'winner_class']] = pred_team_matches[['de_overpass','de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2','de_cbble', 'de_nuke', 'winner_class']]
std_pred_team_matches

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,-0.114617,-0.323967,1.319312,1,0,0,0,0,0,0,0,-1
1,3.102073,1.318208,2.518032,0,1,0,0,0,0,0,0,-1
2,0.804437,0.867434,0.008996,0,0,1,0,0,0,0,0,-1
3,1.034201,0.910688,0.919341,0,0,0,1,0,0,0,0,1
4,0.804437,1.987434,-0.392544,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,0.804437,-0.679048,0.313558,0,1,0,0,0,0,0,0,1
13457,-0.344381,-1.651014,-0.068817,0,0,0,0,0,1,0,0,1
13458,-1.722962,0.051237,-1.687732,0,0,1,0,0,0,0,0,1
13459,-0.344381,-1.555245,0.112620,0,0,0,0,1,0,0,0,1


In [14]:
pred_team_rounds.columns

Index(['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val',
       't_eq_val', 'is_bomb_planted', 'bomb_site', 'past_halftime',
       'de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train',
       'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO',
       'NORMAL', 'FORCE_BUY', 't_lethal_nades_eff', 'ct_lethal_nades_eff',
       't_util_nades_eff', 'ct_util_nades_eff', 'winner_side_class'],
      dtype='object')

In [15]:
scaled_cols = ['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val','t_eq_val','t_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff', 'ct_util_nades_eff']
unscaled_cols = ['is_bomb_planted','bomb_site','de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY','winner_side_class']

std_pred_team_rounds = pd.DataFrame(data=StandardScaler().fit_transform(pred_team_rounds[scaled_cols]),columns=scaled_cols)
std_pred_team_rounds[unscaled_cols] = pred_team_rounds[unscaled_cols]
std_pred_team_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,is_bomb_planted,bomb_site,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,winner_side_class
0,-1.584582,-1.581223,-1.606967,-0.734685,-1.399625,-1.609951,-1.061526,-0.851189,-1.498029,-0.993525,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1
1,-1.459545,-1.490558,-1.446113,1.091929,-1.209110,0.119316,0.282583,-0.389366,-1.140547,-0.615697,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1
2,-1.334508,-1.329444,-1.362443,-0.912541,-1.080513,0.102194,-0.613489,-0.851189,-1.140547,-1.371353,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1
3,-1.209471,-1.245639,-1.234214,0.244630,0.562679,0.575888,-0.165453,1.457926,1.361827,-0.993525,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1
4,-1.084434,-1.117204,-1.167920,-1.363810,-1.294842,0.250580,-0.613489,-0.851189,-0.783065,-0.993525,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,0.541048,0.531166,0.475943,-1.411950,-1.380574,0.410380,-0.613489,-0.851189,-0.425583,-1.371353,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,-1
340329,0.666085,0.595711,0.555776,-1.012200,-0.247010,0.884074,0.282583,-0.389366,0.646863,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1
340330,0.791123,0.675672,0.623980,-1.314217,-0.370845,0.872660,-1.061526,-0.620278,-0.068101,0.139959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-1
340331,0.916160,0.743986,0.695181,-1.236376,-0.137464,0.815588,0.282583,-0.158455,-0.783065,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,-1


In [16]:
pred_side_rounds.columns

Index(['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val',
       't_eq_val', 'is_bomb_planted', 'bomb_site', 'past_halftime',
       'de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train',
       'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO',
       'NORMAL', 'FORCE_BUY', 't_lethal_nades_eff', 'ct_lethal_nades_eff',
       't_util_nades_eff', 'ct_util_nades_eff', 'winner_class'],
      dtype='object')

In [17]:
scaled_cols = ['round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val','t_eq_val','t_lethal_nades_eff', 'ct_lethal_nades_eff', 't_util_nades_eff', 'ct_util_nades_eff']
unscaled_cols = ['is_bomb_planted','bomb_site','de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2', 'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL', 'FORCE_BUY','winner_class']

std_pred_side_rounds = pd.DataFrame(data=StandardScaler().fit_transform(pred_side_rounds[scaled_cols]),columns=scaled_cols)
std_pred_side_rounds[unscaled_cols] = pred_side_rounds[unscaled_cols]
std_pred_side_rounds

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,is_bomb_planted,bomb_site,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,winner_class
0,-1.584582,-1.581223,-1.606967,-0.734685,-1.399625,-1.609951,-1.061526,-0.851189,-1.498029,-0.993525,1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,-1
1,-1.459545,-1.490558,-1.446113,1.091929,-1.209110,0.119316,0.282583,-0.389366,-1.140547,-0.615697,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,-1
2,-1.334508,-1.329444,-1.362443,-0.912541,-1.080513,0.102194,-0.613489,-0.851189,-1.140547,-1.371353,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-1
3,-1.209471,-1.245639,-1.234214,0.244630,0.562679,0.575888,-0.165453,1.457926,1.361827,-0.993525,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,-1
4,-1.084434,-1.117204,-1.167920,-1.363810,-1.294842,0.250580,-0.613489,-0.851189,-0.783065,-0.993525,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,0.541048,0.531166,0.475943,-1.411950,-1.380574,0.410380,-0.613489,-0.851189,-0.425583,-1.371353,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
340329,0.666085,0.595711,0.555776,-1.012200,-0.247010,0.884074,0.282583,-0.389366,0.646863,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
340330,0.791123,0.675672,0.623980,-1.314217,-0.370845,0.872660,-1.061526,-0.620278,-0.068101,0.139959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
340331,0.916160,0.743986,0.695181,-1.236376,-0.137464,0.815588,0.282583,-0.158455,-0.783065,-0.237869,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1


### set up hyperparameter search and train test split

In [18]:
def ttSplit(df,test_size,random_state):

  data = df.iloc[:, :-1]
  target = df.iloc[:,-1:]
  x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=test_size, random_state=random_state)

  return x_train, x_test, y_train, y_test

In [19]:
def hyperparameter_exhaustive_search (model, parameters, scoring, x_train, x_test, y_train, y_test):
  for i in range (len(scoring)):

        startTime = time.time()

        clf = GridSearchCV(estimator=model, param_grid=parameters, scoring=scoring[i],n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        endTime = time.time()

        clf_report = classification_report(y_test,y_pred)

        print('Optimizing hyperparameters for ', scoring[i])
        print('\nElapsed Time (s):', round((endTime -startTime),3))
        
        print("\nBest parameters:", clf.best_params_)

        print('\nClassification report:\n', clf_report)

        print('-----------------------------------------------------------------------------\n')

  print('Search Complete!')

In [20]:
def hyperparameter_random_search (model, parameters, scoring, x_train, x_test, y_train, y_test):
    for i in range (len(scoring)):

        startTime = time.time()

        clf = RandomizedSearchCV(estimator=model, param_distributions=parameters, scoring=scoring[i],n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        endTime = time.time()

        clf_report = classification_report(y_test,y_pred)

        print('Optimizing hyperparameters for ', scoring[i])
        print('\nElapsed Time (s):', round((endTime -startTime),3))
        
        print("\nBest parameters:", clf.best_params_)

        print('\nClassification report:\n', clf_report)

        print('-----------------------------------------------------------------------------\n')

### XGBoost

In [21]:
def find_best_XGB(x_train, x_test, y_train, y_test):
  model = XGBClassifier()
  xgbparameters = {'learning_rate': [0.1, 0.2, 1], 'max_depth': [1,2,3,4], 'n_estimators': [10, 50, 100,200]}

  #the average function is binary by default. This causes error in the GridSearchCV function for Decision Trees. therefore pass a score function with custom average settings
  average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
  scoring =[
      make_scorer(f1_score,average=average_method)
      ]

  hyperparameter_exhaustive_search(model=model, parameters=xgbparameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [22]:
def find_random_best_XGB(x_train, x_test, y_train, y_test):
  model = XGBClassifier()
  xgbparameters = {'learning_rate': [0.1, 0.2, 1], 'max_depth': [1,2,3,4], 'n_estimators': [10, 50, 100,200]}

  #the average function is binary by default. This causes error in the GridSearchCV function for Decision Trees. therefore pass a score function with custom average settings
  average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
  scoring =[
      make_scorer(f1_score,average=average_method)
      ]

  hyperparameter_random_search(model=model, parameters=xgbparameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [25]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_team_matches[:50000],test_size=0.2,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_XGB(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 194.363

Best parameters: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}

Classification report:
               precision    recall  f1-score   support

          -1       0.51      0.35      0.41      1257
           0       0.32      0.16      0.21       106
           1       0.55      0.73      0.63      1330

    accuracy                           0.53      2693
   macro avg       0.46      0.41      0.42      2693
weighted avg       0.52      0.53      0.51      2693

-----------------------------------------------------------------------------

Search Complete!


In [26]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_team_rounds[:50000],test_size=0.2,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_XGB(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 452.383

Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}

Classification report:
               precision    recall  f1-score   support

          -1       0.78      0.83      0.81      5076
           1       0.82      0.76      0.79      4924

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000

-----------------------------------------------------------------------------

Search Complete!


In [27]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_side_rounds[:50000],test_size=0.2,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_XGB(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 444.638

Best parameters: {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100}

Classification report:
               precision    recall  f1-score   support

          -1       0.79      0.78      0.79      4898
           1       0.79      0.80      0.80      5102

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000

-----------------------------------------------------------------------------

Search Complete!


### ADABoost

In [40]:
def find_best_ADA(x_train, x_test, y_train, y_test):
  model = AdaBoostClassifier()
  
  adaparameters = {'learning_rate': [0.1, 0.2, 0.5, 1, 1.5], 'n_estimators': [10, 50, 100,200], 'base_estimator':[DecisionTreeClassifier(max_depth=2),DecisionTreeClassifier(max_depth=3)]}

  average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
  scoring =[
      make_scorer(f1_score,average=average_method)
      ]

  hyperparameter_exhaustive_search(model=model, parameters=adaparameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [41]:
def find_random_best_ADA(x_train, x_test, y_train, y_test):
  model = AdaBoostClassifier()
  
  adaparameters = {'learning_rate': [0.1, 0.2, 0.5, 1, 1.5], 'n_estimators': [2, 5,10, 50], 'base_estimator':[DecisionTreeClassifier(max_depth=2),DecisionTreeClassifier(max_depth=3)]}

  average_method = 'micro'  # None, ‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’
  scoring =[
      make_scorer(f1_score,average=average_method)
      ]

  hyperparameter_random_search(model=model, parameters=adaparameters, scoring=scoring, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [42]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_side_rounds[:10000],test_size=0.2,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_ADA(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 248.126

Best parameters: {'base_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), 'learning_rate': 0.2, 'n_estimators': 50}

Classification report:
               precision    recall  f1-score   support

          -1       0.79      0.76      0.77      1043
           1       0.75      0.77      0.76       957

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000

------------------------------------------------

In [43]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_team_matches[:10000],test_size=0.2,random_state=0) #only the first 50k lines. takes forver otherwise
find_best_ADA(x_train, x_test, y_train, y_test)

Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 139.599

Best parameters: {'base_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), 'learning_rate': 0.1, 'n_estimators': 10}

Classification report:
               precision    recall  f1-score   support

          -1       0.45      0.36      0.40       887
           0       0.00      0.00      0.00        79
           1       0.55      0.69      0.61      1034

    accuracy                           0.52      2000
   macro avg       0.33      0.35      0.34      2000
weighted avg       0.48      0.52      0.49      

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
x_train, x_test, y_train, y_test = ttSplit(df=std_pred_side_rounds[:100000],test_size=0.2,random_state=0) # first 100k lines with random search
find_random_best_ADA(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimizing hyperparameters for  make_scorer(f1_score, average=micro)

Elapsed Time (s): 94.4

Best parameters: {'n_estimators': 10, 'learning_rate': 1, 'base_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')}

Classification report:
               precision    recall  f1-score   support

          -1       0.77      0.77      0.77      9846
           0       0.00      0.00      0.00         1
           1       0.78      0.78      0.78     10153

    accuracy                           0.77     20000
   macro avg       0.52      0.52      0.52     20000
weighted avg       0.77      0.77      0.77     20000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
